In [60]:
# Install necessary packages
!pip install transformers sentencepiece torch torchvision albumentations timm


In [61]:
# Import required libraries
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
import numpy as np
import pandas as pd
import itertools
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import cv2
import timm

In [62]:

# Step 1: Mount Google Drive to access the dataset.
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
# Define paths to the image folder and caption file
image_folder = '/content/drive/MyDrive/Bangla Image dataset with caption/BNATURE/Pictures'
caption_file = '/content/drive/MyDrive/Bangla Image dataset with caption/BNATURE/caption/captions.json'


In [64]:
# Define configuration with adjusted batch size
class CFG:
    model_name = "resnet50"
    text_encoder_model = "csebuetnlp/banglabert"  # Bangla BERT model
    pretrained = True
    trainable = True
    batch_size = 16  # Reduced batch size
    size = 224
    image_embedding = 2048
    text_embedding = 768
    projection_dim = 512
    max_length = 128
    temperature = 0.07
    image_encoder_lr = 1e-4
    text_encoder_lr = 1e-5
    head_lr = 1e-6
    weight_decay = 1e-4
    patience = 2
    factor = 0.5
    device = "cuda" if torch.cuda.is_available() else "cpu"
    epochs = 15



In [65]:


# Dataset class
class CLIPDataset(Dataset):
    def __init__(self, image_filenames, captions, tokenizer, transforms):
        self.image_filenames = image_filenames
        self.captions = list(captions)
        self.encoded_captions = tokenizer(
            list(captions), padding=True, truncation=True, max_length=CFG.max_length
        )
        self.transforms = transforms

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }
        image = cv2.imread(f"{image_folder}/{self.image_filenames[idx]}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']
        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        item['caption'] = self.captions[idx]
        return item

    def __len__(self):
        return len(self.captions)

In [66]:

# Image Encoder
class ImageEncoder(nn.Module):
    def __init__(self, model_name=CFG.model_name, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained, num_classes=0, global_pool="avg")
        for p in self.model.parameters():
            p.requires_grad = trainable

    def forward(self, x):
        return self.model(x)

In [67]:
# Text Encoder
class TextEncoder(nn.Module):
    def __init__(self, model_name=CFG.text_encoder_model, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)
        for p in self.model.parameters():
            p.requires_grad = trainable
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = output.last_hidden_state
        return last_hidden_state[:, self.target_token_idx, :]

In [68]:
# Projection Head
class ProjectionHead(nn.Module):
    def __init__(self, embedding_dim, projection_dim=CFG.projection_dim, dropout=0.1):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)

    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

In [69]:

# CLIP Model
class CLIPModel(nn.Module):
    def __init__(self, temperature=CFG.temperature):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectionHead(embedding_dim=CFG.image_embedding)
        self.text_projection = ProjectionHead(embedding_dim=CFG.text_embedding)
        self.temperature = temperature

    def forward(self, batch):
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)
        logits = (text_embeddings @ image_embeddings.T) / self.temperature
        images_similarity = image_embeddings @ image_embeddings.T
        texts_similarity = text_embeddings @ text_embeddings.T
        targets = F.softmax((images_similarity + texts_similarity) / 2 * self.temperature, dim=-1)
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss = (images_loss + texts_loss) / 2.0
        return loss.mean()

In [70]:

# Cross Entropy
def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    return loss.mean() if reduction == "mean" else loss

# Data loaders
def get_transforms():
    return A.Compose([
        A.Resize(CFG.size, CFG.size, always_apply=True),
        A.Normalize(max_pixel_value=255.0, always_apply=True),
    ])

In [71]:
with open(caption_file, 'r', encoding='utf-8') as f:
    captions_data = json.load(f)

# Print a sample item to inspect the structure
print(captions_data[0])  # Check the first item for key names


{'caption_id': '1.jpg', 'bengali_caption': '  গ্রামে হাঁটা দুই শিশু।'}


In [72]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  # Add this line for functional operations
import albumentations as A
import numpy as np
import pandas as pd
import itertools
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import cv2
import timm


In [73]:
import zipfile
import os

# Path to the zip file
zip_file_path = '/content/drive/MyDrive/Bangla Image dataset with caption/final_clip_model.zip'
extract_path = '/content/drive/MyDrive/Bangla Image dataset with caption/'

# Unzip the model
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Model unzipped successfully.")


Model unzipped successfully.


In [75]:
import torch
import json
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import os  # For checking and creating directories if needed

# Optimized train_epoch with mixed precision
def train_epoch(model, train_loader, optimizer, lr_scheduler, step):
    model.train()
    scaler = torch.cuda.amp.GradScaler()  # Mixed precision scaler
    loss_meter = 0
    for batch in tqdm(train_loader, total=len(train_loader)):
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}

        with torch.cuda.amp.autocast():  # Mixed precision training
            loss = model(batch)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if step == "batch":
            lr_scheduler.step()

        loss_meter += loss.item()
    return loss_meter / len(train_loader)

# Validation function remains the same
def valid_epoch(model, valid_loader):
    model.eval()
    loss_meter = 0
    with torch.no_grad():
        for batch in tqdm(valid_loader, total=len(valid_loader)):
            batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}
            loss = model(batch)
            loss_meter += loss.item()
    return loss_meter / len(valid_loader)

# Main function with optimizations for data loading and mixed precision training
def main():
    train_losses = []
    valid_losses = []

    with open(caption_file, 'r', encoding='utf-8') as f:
        captions_data = json.load(f)

    # Extract image files and captions based on provided structure
    image_files = [item['caption_id'].split('#')[0] for item in captions_data]
    captions = [item['bengali_caption'] for item in captions_data]

    tokenizer = AutoTokenizer.from_pretrained(CFG.text_encoder_model)
    transforms = get_transforms()
    dataset = CLIPDataset(image_files, captions, tokenizer, transforms)

    # Data splitting into train and validation sets
    train_size = int(0.8 * len(dataset))
    valid_size = len(dataset) - train_size
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

    # DataLoader with num_workers for faster data loading
    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4)

    # Initialize the model
    model = CLIPModel().to(CFG.device)

    # Load pre-trained weights if they exist
    pre_trained_model_path = "/content/drive/MyDrive/Bangla Image dataset with caption/final_clip_model.pt"
    if os.path.exists(pre_trained_model_path):
        model.load_state_dict(torch.load(pre_trained_model_path, map_location=CFG.device))
        print(f"Loaded pre-trained model from {pre_trained_model_path}")
    else:
        print(f"No pre-trained model found at {pre_trained_model_path}. Starting training from scratch.")

    model.train()  # Set model to training mode if fine-tuning

    # Define optimizer and learning rate scheduler
    params = [
        {"params": model.image_encoder.parameters(), "lr": CFG.image_encoder_lr},
        {"params": model.text_encoder.parameters(), "lr": CFG.text_encoder_lr},
        {"params": itertools.chain(model.image_projection.parameters(), model.text_projection.parameters()),
         "lr": CFG.head_lr, "weight_decay": CFG.weight_decay}
    ]

    optimizer = torch.optim.AdamW(params, weight_decay=0.0)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=CFG.patience, factor=CFG.factor)
    step = "epoch"

    # Variable to track best validation loss
    best_valid_loss = float('inf')

    # Start training loop
    for epoch in range(CFG.epochs):
        print(f"Epoch {epoch + 1}/{CFG.epochs}")

        train_loss = train_epoch(model, train_loader, optimizer, lr_scheduler, step)
        print(f"Train Loss: {train_loss:.4f}")
        train_losses.append(train_loss)  # Store the train loss

        valid_loss = valid_epoch(model, valid_loader)
        print(f"Validation Loss: {valid_loss:.4f}")
        valid_losses.append(valid_loss)  # Store the validation loss

        if step == "epoch":
            lr_scheduler.step(valid_loss)

        # Save the model after each epoch in Google Drive
        model_save_path = f"/content/drive/MyDrive/Bangla Image dataset with caption/epoch_{epoch+1}_clip_model.pt"
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved to {model_save_path}")

        # Save the model if it achieves the best validation loss so far
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            print(f"Validation loss decreased. Saving the best model...")
            torch.save(model.state_dict(), "/content/drive/MyDrive/Bangla Image dataset with caption/best_clip_model.pt")

    # Plotting the train and validation loss after the training loop
    plot_losses(train_losses, valid_losses)

    # Optionally save the final model
    final_model_path = "/content/drive/MyDrive/Bangla Image dataset with caption/final_clip_model.pt"
    torch.save(model.state_dict(), final_model_path)  # Save the final model after training

def plot_losses(train_losses, valid_losses):
    """Plot train and validation losses"""
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss', color='blue', linestyle='-', marker='o')
    plt.plot(valid_losses, label='Validation Loss', color='red', linestyle='-', marker='x')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

# Run the training without early stopping
main()


<ipython-input-75-a46b4766ce3c>:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(pre_trained_model_path, map_location=CFG.device))
<ipython

Loaded pre-trained model from /content/drive/MyDrive/Bangla Image dataset with caption/final_clip_model.pt
Epoch 1/15


  0%|          | 0/1956 [00:00<?, ?it/s]<ipython-input-75-a46b4766ce3c>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Mixed precision training
100%|██████████| 1956/1956 [05:24<00:00,  6.03it/s]


Train Loss: 1.3935


100%|██████████| 489/489 [00:45<00:00, 10.84it/s]


...

Validation Loss: 1.3981
Model saved to /content/drive/MyDrive/Bangla Image dataset with caption/epoch_3_clip_model.pt
Epoch 4/15


100%|██████████| 1956/1956 [05:21<00:00,  6.08it/s]


Train Loss: 1.3944


 56%|█████▌    | 275/489 [00:25<00:19, 10.76it/s]


KeyboardInterrupt: 

#iNTERFACE

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm
import json

# Assuming CLIPDataset and CFG have already been defined
# Ensure the paths to the dataset are properly set

# Load and prepare the dataset
with open(caption_file, 'r', encoding='utf-8') as f:
    captions_data = json.load(f)

# Prepare image files and captions
image_files = [item['caption_id'].split('#')[0] for item in captions_data]
captions = [item['bengali_caption'] for item in captions_data]

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(CFG.text_encoder_model)
transforms = get_transforms()

# Create the dataset
dataset = CLIPDataset(image_files, captions, tokenizer, transforms)

# Split dataset into train and validation sets
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
_, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

# Define the function to get image embeddings
def get_image_embeddings(valid_dataset, model_path):
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4)

    # Load the trained model
    model = CLIPModel().to(CFG.device)
    model.load_state_dict(torch.load(model_path, map_location=CFG.device))  # Load final model
    model.eval()

    valid_image_embeddings = []
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            # Move image tensors to the correct device
            images = batch["image"].to(CFG.device)

            # Get image features from the encoder
            image_features = model.image_encoder(images)

            # Project image features into the desired embedding space
            image_embeddings = model.image_projection(image_features)

            # Append the image embeddings to the list
            valid_image_embeddings.append(image_embeddings)

    # Concatenate all image embeddings into a single tensor and return
    return model, torch.cat(valid_image_embeddings)

# Perform inference to get image embeddings from the validation set
model, image_embeddings = get_image_embeddings(valid_dataset, "/content/final_clip_model.pt")  # Use final_clip_model.pt

# Example usage of the image embeddings
print(f"Image Embeddings shape: {image_embeddings.shape}")

In [ ]:
import matplotlib.pyplot as plt  # Add this import


In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import cv2

def find_matches(model, image_embeddings, query, image_files, n=1):
    tokenizer = AutoTokenizer.from_pretrained(CFG.text_encoder_model)

    # Encode the query
    encoded_query = tokenizer([query], return_tensors="pt", padding=True, truncation=True, max_length=CFG.max_length)
    print(f"Encoded Query: {encoded_query}")  # For debugging

    # Prepare the batch for the model
    batch = {
        key: torch.tensor(values).to(CFG.device)
        for key, values in encoded_query.items()
    }

    with torch.no_grad():
        # Get text features from the model
        text_features = model.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        # Project text features into the same space as image features
        text_embeddings = model.text_projection(text_features)

    # Normalize the embeddings
    text_embeddings_n = F.normalize(text_embeddings, p=2, dim=-1)
    image_embeddings_n = F.normalize(image_embeddings, p=2, dim=-1)

    # Calculate cosine similarity between the query and all image embeddings
    dot_similarity = text_embeddings_n @ image_embeddings_n.T
    values, indices = torch.topk(dot_similarity.squeeze(0), n)

    # Get the top match (most similar image)
    top_match = image_files[indices[0]]  # The most similar image file
    top_similarity_score = values[0].item()  # The similarity score for the top match

    # Output the results
    print(f"Top match: {top_match} with similarity score: {top_similarity_score:.4f}")

    # Read and display the top matching image
    image = cv2.imread(f"{image_folder}/{top_match}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Plot the image with the similarity score in the title
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.axis("off")  # Hide axes
    plt.title(f"Similarity Score: {top_similarity_score:.4f}")  # Display similarity score
    plt.show()

# Example usage with the query prompt
prompt = "কুকুর"  # Example prompt for "A dog"
find_matches(model, image_embeddings, prompt, image_files)


In [ ]:
import zipfile
import os

# Update model path to the final model
model_path = '/content/final_clip_model.pt'
zip_path = '/content/final_clip_model.zip'

# Creating a zip file
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(model_path, os.path.basename(model_path))

print(f"Model has been zipped and saved as {zip_path}")


In [ ]:
from google.colab import files

# Trigger file download
files.download(zip_path)


In [ ]:
!pip install huggingface_hub


In [ ]:
!huggingface-cli login


In [ ]:
!git config --global user.name "Mansuba"
!git config --global user.email "mansubatabassum9@gmail.com"

In [ ]:
from huggingface_hub import HfApi, Repository

# Ensure you're logged in to Hugging Face Hub
# This is already assumed to be done; if not, run `huggingface-cli login`

# Define the model path and repo information
model_path = '/content/final_clip_model.zip'  # Path to your model zip file
repo_name = "Mansuba/BanglaClip"  # The repo where the model will be uploaded

# Initialize the repository (clone from Hugging Face Hub)
repo = Repository(local_dir='/content/BanglaClipRepo', clone_from=repo_name)

# Step 1: Copy the model to the repo directory
import shutil
shutil.copy(model_path, '/content/BanglaClipRepo')  # Copy model file into the repo directory

# Step 2: Push the model to Hugging Face Hub
repo.push_to_hub(commit_message="Add final_clip_model.zip")

print("Model uploaded successfully to Hugging Face Hub!")


In [ ]:
# Check the status of the git repo to ensure everything is up-to-date
!git -C /content/BanglaClipRepo status

# Push to Hugging Face Hub manually from the local repo
!git -C /content/BanglaClipRepo push origin main


In [ ]:
!git -C /content/BanglaClipRepo status


In [ ]:
!git -C /content/BanglaClipRepo add final_clip_model.zip


In [ ]:
!git -C /content/BanglaClipRepo commit -m "Add final_clip_model.zip"


In [ ]:
!git -C /content/BanglaClipRepo push origin main


In [ ]:
from google.colab import drive
import shutil

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the path for the model in Google Drive
model_path = '/content/final_clip_model.zip'  # Path to the model zip file
drive_folder = '/content/drive/MyDrive/Bangla Image dataset with caption'  # Target directory in Google Drive

# Step 3: Create the target folder if it doesn't exist
import os
if not os.path.exists(drive_folder):
    os.makedirs(drive_folder)

# Step 4: Copy the model to the target folder
shutil.copy(model_path, os.path.join(drive_folder, 'final_clip_model.zip'))

print(f"Model saved to Google Drive at: {os.path.join(drive_folder, 'final_clip_model.zip')}")
